# SERAB demo
* This notebook decomposes `clf_benchmark.py`, which evaluate a given pre-trained model against SERAB to show the main steps behind the script.



In [81]:
!git clone https://github.com/Neclow/SERAB.git

fatal: destination path 'SERAB' already exists and is not an empty directory.


In [82]:
!git clone https://github.com/Neclow/SERAB.git
!cd SERAB/SERAB/ && curl -O https://raw.githubusercontent.com/nttcslab/byol-a/f2451c366d02be031a31967f494afdf3485a85ff/config.yaml && patch --ignore-whitespace < config.diff
!cd SERAB/SERAB/ && curl -O https://raw.githubusercontent.com/nttcslab/byol-a/f2451c366d02be031a31967f494afdf3485a85ff/train.py && patch < train.diff
!cd SERAB/SERAB/byol_a/ && curl -O https://raw.githubusercontent.com/nttcslab/byol-a/f2451c366d02be031a31967f494afdf3485a85ff/byol_a/augmentations.py && patch < augmentations.diff
!cd SERAB/SERAB/byol_a/ &&curl -O https://raw.githubusercontent.com/nttcslab/byol-a/f2451c366d02be031a31967f494afdf3485a85ff/byol_a/common.py && patch < common.diff
!cd SERAB/SERAB/byol_a/ &&curl -O https://raw.githubusercontent.com/nttcslab/byol-a/f2451c366d02be031a31967f494afdf3485a85ff/byol_a/dataset.py && patch < dataset.diff
!cd SERAB/SERAB/byol_a/ && curl -O https://raw.githubusercontent.com/nttcslab/byol-a/f2451c366d02be031a31967f494afdf3485a85ff/byol_a/models.py && mv models.py models/audio_ntt.py

fatal: destination path 'SERAB' already exists and is not an empty directory.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   564  100   564    0     0   3296      0 --:--:-- --:--:-- --:--:--  3317
patching file config.yaml
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4867  100  4867    0     0  22998      0 --:--:-- --:--:-- --:--:-- 23066
patching file train.py
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  8273  100  8273    0     0  42926      0 --:--:-- --:--:-- --:--:-- 42865
patching file augmentations.py
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   To

## Install libraries

In [1]:
!apt-get install libsox-fmt-all libsox-dev sox > /dev/null
!pip install -q opensmile

!pip install -q pytorch-lightning==1.4.0 torchmetrics==0.6.0
!pip install -q torch==1.11.0 torchvision==0.12.0 torchaudio==0.11.0 torchtext==0.12.0
!pip install -q einops==0.4.0
!pip install -q pydub==0.25.1
!pip install -q tensorflow_datasets==4.3.0
!pip install -q librosa==0.8.1
!pip install --upgrade librosa
!pip install torch==1.12.1 torchtext==0.13.1 pytorch-lightning==1.9.3
!pip install torchaudio==0.12.1

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.13.2 requires torch>=1.13.0, but you have torch 1.11.0 which is incompatible.
  Using cached librosa-0.10.2.post1-py3-none-any.whl.metadata (8.6 kB)
Using cached librosa-0.10.2.post1-py3-none-any.whl (260 kB)
  Attempting uninstall: librosa
    Found existing installation: librosa 0.8.1
    Uninstalling librosa-0.8.1:
      Successfully uninstalled librosa-0.8.1
  Using cached torch-1.12.1-cp310-cp310-manylinux1_x86_64.whl.metadata (22 kB)
  Using cached torchtext-0.13.1-cp310-cp310-manylinux1_x86_64.whl.metadata (6.9 kB)
  Using cached pytorch_lightning-1.9.3-py3-none-any.whl.metadata (22 kB)
  Using cached torchmetrics-1.5.1-py3-none-any.whl.metadata (20 kB)
Using cached torch-1.12.1-cp310-cp310-manylinux1_x86_64.whl (776.3 MB)
Using cached torchtext-0.13.1-cp310-cp310-manylinux1_x86_64.whl (1.9 MB)
Using

## Load imports and config variables

In [2]:
SERAB_PATH = 'SERAB/SERAB/'

import sys

sys.path.append(SERAB_PATH)

In [3]:
import collections
import os

import librosa
import numpy as np
import pandas as pd
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import torch





In [4]:
from pytorch_lightning.utilities.seed import seed_everything
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.metrics import recall_score
from torchaudio.transforms import MelSpectrogram

In [5]:
from byol_a.common import load_yaml_config
from byol_a.augmentations import PrecomputedNorm
from clf_benchmark import dat_from_split, get_sklearn_models
from settings import CLF_STATS_DICT, RANDOM_SEED, REQUIRED_SAMPLE_RATE
from utils import compute_norm_stats, generate_embeddings, load_model, save_results, speaker_normalization

In [6]:
seed_everything(RANDOM_SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cfg = load_yaml_config(f'{SERAB_PATH}/config.yaml')
to_melspec = MelSpectrogram(
    sample_rate=cfg.sample_rate,
    n_fft=cfg.n_fft,
    win_length=cfg.win_length,
    hop_length=cfg.hop_length,
    n_mels=cfg.n_mels,
    f_min=cfg.f_min,
    f_max=cfg.f_max,
)
results = {}

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/seed.py:47: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v2.0.0. Please use `lightning_fabric.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
INFO:lightning_fabric.utilities.seed:Global seed set to 42


## Load data
* If you want to test a model on a dataset other than CREMA-D you will need to download and build the dataset.
    * An example of how to proceed with emoDB is shown below. Uncomment the lines below (`UNCOMMENT CODE HERE`) to build the emodB dataset.
        * You will need to create the subfolders `downloads/manual` under `tensorflow_datasets` and paste the compressed emodB dataset in there.
* For this demo, we will use CREMA-D as already pre-built in TFDS.

In [7]:
# Create tensorflow_datasets folders
!mkdir -p ../root/tensorflow_datasets/
!mkdir -p ../root/tensorflow_datasets/downloads/
!mkdir -p ../root/tensorflow_datasets/downloads/manual

## UNCOMMENT CODE HERE
## Copy compressed dataset files into download/manual
# !cp -n drive/MyDrive/SERAB/tensorflow_datasets/downloads/manual/*.zip ../root/tensorflow_datasets/downloads/manual/

## Copy the emoDB TFDS scripts and build the emoDB dataset
# !cp -r -n SERAB/tensorflow_datasets/emoDB ../root/tensorflow_datasets/emoDB
# !cd ../root/tensorflow_datasets/emoDB && tfds build emoDB

In [8]:
dataset_name = 'crema_d'

In [9]:
model_selection = 'predefined'

SingleSplit = collections.namedtuple('SingleSplit', ['audio', 'labels', 'speaker_id'])
Data = collections.namedtuple('Data', ['train', 'validation', 'test'])
all_data = Data(
    train=SingleSplit(*dat_from_split(dataset_name, 'train')),
    validation=SingleSplit(*dat_from_split(dataset_name, 'validation')),
    test=SingleSplit(*dat_from_split(dataset_name, 'test'))
)

orig_sr = tfds.builder(dataset_name).info.features['audio'].sample_rate
num_classes = len(np.unique(all_data.train.labels))

Finished train
Finished validation
Finished test


In [10]:
# Load data statistics
try:
    stats = CLF_STATS_DICT[dataset_name]
except KeyError:
    print(f'Did not find mean/std stats for {dataset_name}.')
    stats = compute_norm_stats(dataset_name, all_data.train.audio, orig_sr, to_melspec)

    CLF_STATS_DICT[dataset_name] = stats

    print(CLF_STATS_DICT)
normalizer = PrecomputedNorm(stats)

## Load the model
* For this demo, we will use "BYOL-S", a re-trained BYOL-A on speech samples of AudioSet with 512 features.
    * More information regarding the models can be found in the pre-print (https://arxiv.org/abs/2110.03414)

In [11]:
model_name = 'default'
ckpt_folder = f'{SERAB_PATH}/checkpoints/'

model, weight_file = load_model(model_name, cfg, device, ckpt_folder)

print(weight_file)

SERAB/SERAB/checkpoints/default1024_BYOLAs64x96-2107292000-e100-bs256-lr0003-rs42.pth


In [12]:
# Generate embeddings
embeddings = Data(
    train=generate_embeddings(
        model,
        model_name,
        all_data.train.audio,
        'train',
        orig_sr,
        to_melspec,
        normalizer,
        device
    ),
    validation=generate_embeddings(
        model,
        model_name,
        all_data.validation.audio,
        'validation',
        orig_sr,
        to_melspec,
        normalizer,
        device
    ),
    test=generate_embeddings(
        model,
        model_name,
        all_data.test.audio,
        'test',
        orig_sr,
        to_melspec,
        normalizer,
        device
    )
)

Generating embeddings for train: 100%|##########| 5144/5144 [02:26<00:00, 35.12it/s]


Finished train


Generating embeddings for validation: 100%|##########| 738/738 [00:20<00:00, 35.27it/s]


Finished validation


Generating embeddings for test: 100%|##########| 1556/1556 [00:42<00:00, 36.64it/s]

Finished test


In [13]:
print(embeddings.train.mean(), embeddings.train.std())
print(embeddings.validation.mean(), embeddings.validation.std())
print(embeddings.test.mean(), embeddings.test.std())
print(embeddings.train.shape)
print(embeddings.validation.shape)
print(embeddings.test.shape)

# Load classifiers
log_list, estimator_list, param_list = get_sklearn_models()

4.344951 4.3818913
4.3764834 4.4149346
4.289814 4.375481
(5144, 1024)
(738, 1024)
(1556, 1024)


In [14]:
# Assuming labels are in a structure like all_data.train.labels
# Define labels_train to match embeddings.train
from sklearn.decomposition import PCA
labels_train = all_data.train.labels  # Adjust this to match your actual data structure

# Step 1: Verify Dimensions of Original Data Before PCA
print("Original shape of embeddings.train:", embeddings.train.shape)
print("Original shape of labels_train:", len(labels_train))

# Step 2: Check for any mismatch in dimensions after PCA transformation
if embeddings.train.shape[0] != len(labels_train):
    print(f"Mismatch detected: embeddings.train has {embeddings.train.shape[0]} samples, but labels_train has {len(labels_train)} samples.")
    # Align labels if needed - adjust if there's a specific filter applied on embeddings.train
    indices_to_keep = np.arange(embeddings.train.shape[0])  # Adjust indices if there's filtering
    labels_train = labels_train[indices_to_keep]

# Step 3: Apply PCA on the embeddings with matched labels
pca = PCA(n_components=0.95)
train_features = pca.fit_transform(embeddings.train)
test_features = pca.transform(embeddings.test)

# Verify the shape after reprocessing
print("New shape of train_features:", train_features.shape)
print("New shape of labels_train:", len(labels_train))

# Proceed with model training
# Now train_features and labels_train should have matching dimensions


Original shape of embeddings.train: (5144, 1024)
Original shape of labels_train: 5144
New shape of train_features: (5144, 228)
New shape of labels_train: 5144


In [15]:
# from sklearn.decomposition import PCA

# pca = PCA(n_components=0.95)
# train_features = pca.fit_transform(embeddings.train)
# test_features = pca.transform(embeddings.test)

# print(f"Original dimensions: {embeddings.train.shape[1]}, Reduced dimensions: {train_features.shape[1]}")


In [16]:
print("Shape of train_features:", train_features.shape)
print("Length of labels_train:", len(labels_train))
print("train_features shape:", train_features.shape)
print("labels_train shape:", labels_train.shape)


Shape of train_features: (5144, 228)
Length of labels_train: 5144
train_features shape: (5144, 228)
labels_train shape: (5144,)


In [17]:
# # Speaker normalization
# # Can also try with normal standardization (StandardScaler), should not change the results too much
# normalized_train = speaker_normalization(embeddings.train, all_data.train.speaker_id)
# normalized_validation = speaker_normalization(embeddings.validation, all_data.validation.speaker_id)
# normalized_test = speaker_normalization(embeddings.test, all_data.test.speaker_id)

# # Aggregate labels and speaker IDs
# normalized_train = np.append(normalized_train, normalized_validation, axis=0)
# labels_train = np.append(all_data.train.labels, all_data.validation.labels, axis=0)
# speaker_id_train = np.append(all_data.train.speaker_id, all_data.validation.speaker_id, axis=0)

In [18]:
# Step 1: Speaker normalization for train, validation, and test embeddings
normalized_train = speaker_normalization(embeddings.train, all_data.train.speaker_id)
normalized_validation = speaker_normalization(embeddings.validation, all_data.validation.speaker_id)
normalized_test = speaker_normalization(embeddings.test, all_data.test.speaker_id)

# Step 2: Combine normalized train and validation embeddings for PCA
combined_embeddings = np.append(normalized_train, normalized_validation, axis=0)

# Step 3: Combine labels from train and validation sets to align with combined_embeddings
labels_train = np.concatenate((all_data.train.labels, all_data.validation.labels), axis=0)

# Step 4: Apply PCA after combining embeddings to match labels_train
pca = PCA(n_components=0.95)
train_features = pca.fit_transform(combined_embeddings)
test_features = pca.transform(normalized_test)

# Verify the shape to ensure train_features and labels_train are aligned
print("normalized_train",normalized_train.shape)
print("normalized_test", normalized_test.shape)
print("normalized_validation", normalized_validation.shape)
print("train_features shape after PCA:", train_features.shape)
print("labels_train shape after alignment:", labels_train.shape)

# Now, train_features and labels_train should match in the number of samples


normalized_train (5144, 1024)
normalized_test (1556, 1024)
normalized_validation (738, 1024)
train_features shape after PCA: (5882, 518)
labels_train shape after alignment: (5882,)


In [19]:
print("train_features shape:", train_features.shape)
print("labels_train shape:", labels_train.shape)


train_features shape: (5882, 518)
labels_train shape: (5882,)


In [20]:
from sklearn.preprocessing import StandardScaler

# Initialize scaler and fit-transform on train features
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)
test_features = scaler.transform(test_features)


In [21]:
# from sklearn.ensemble import StackingClassifier, VotingClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.svm import SVC
# from sklearn.linear_model import LogisticRegression
# from sklearn.preprocessing import StandardScaler

# # Standardize features to improve convergence
# scaler = StandardScaler()
# train_features = scaler.fit_transform(train_features)
# test_features = scaler.transform(test_features)

# # Define base classifiers with increased max_iter and alternative solver for Logistic Regression
# base_estimators = [
#     ('rf', RandomForestClassifier(n_estimators=100)),
#     ('svc', SVC(probability=True)),
#     ('lr', LogisticRegression(max_iter=2000, solver='saga'))
# ]

# # Stacking and Voting Classifiers
# stack_clf = StackingClassifier(estimators=base_estimators, final_estimator=LogisticRegression(max_iter=2000, solver='saga'))
# vote_clf = VotingClassifier(estimators=base_estimators, voting='soft')

# # Fit ensemble models
# stack_clf.fit(train_features, labels_train)
# vote_clf.fit(train_features, labels_train)
from sklearn.ensemble import StackingClassifier, VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# Define base classifiers
base_estimators = [
    ('rf', RandomForestClassifier(n_estimators=100)),
    ('svc', SVC(probability=True)),
    ('lr', LogisticRegression(max_iter=2000))
]

# Stacking and Voting Ensembles
stack_clf = StackingClassifier(estimators=base_estimators, final_estimator=LogisticRegression())
vote_clf = VotingClassifier(estimators=base_estimators, voting='soft')

# Fit both ensemble models
stack_clf.fit(train_features, labels_train)
vote_clf.fit(train_features, labels_train)

# Evaluate initial accuracy on the test set
stack_accuracy = stack_clf.score(test_features, all_data.test.labels)
vote_accuracy = vote_clf.score(test_features, all_data.test.labels)

print(f"Stacking Classifier Accuracy: {stack_accuracy:.2f}")
print(f"Voting Classifier Accuracy: {vote_accuracy:.2f}")




Stacking Classifier Accuracy: 0.70
Voting Classifier Accuracy: 0.68


In [22]:
from sklearn.model_selection import RandomizedSearchCV


param_grid = {
    'rf__n_estimators': [50, 100, 200],
    'svc__C': [0.1, 1, 10],
    'lr__C': [0.1, 1, 10]
}

from sklearn.experimental import enable_halving_search_cv  # Required to enable HalvingRandomSearchCV
from sklearn.model_selection import HalvingRandomSearchCV

search = HalvingRandomSearchCV(vote_clf, param_grid, factor=2, scoring='accuracy', n_jobs=-1)
search.fit(train_features, labels_train)


# Get the best model after tuning
best_model = search.best_estimator_

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=98. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [23]:
from sklearn.metrics import f1_score, precision_score, recall_score

# Predict using the best model
y_pred = best_model.predict(test_features)

# Calculate additional metrics
test_accuracy = best_model.score(test_features, all_data.test.labels)
test_precision = precision_score(all_data.test.labels, y_pred, average='weighted')
test_recall = recall_score(all_data.test.labels, y_pred, average='weighted')
test_f1 = f1_score(all_data.test.labels, y_pred, average='weighted')

print(f"Best Model Test Accuracy: {test_accuracy:.2f}")
print(f"Best Model Test Precision: {test_precision:.2f}")
print(f"Best Model Test Recall: {test_recall:.2f}")
print(f"Best Model Test F1 Score: {test_f1:.2f}")

Best Model Test Accuracy: 0.68
Best Model Test Precision: 0.68
Best Model Test Recall: 0.68
Best Model Test F1 Score: 0.68


In [24]:
import joblib

# Save the best model for future use
joblib.dump(best_model, 'best_audio_classifier.pkl')

# To load the model later
loaded_model = joblib.load('best_audio_classifier.pkl')
print("Model loaded successfully.")


Model loaded successfully.


In [25]:
min_samples = min(train_features.shape[0], labels_train.shape[0])
train_features = train_features[:min_samples]
labels_train = labels_train[:min_samples]


In [26]:
print("train_features shape:", train_features.shape)
print("labels_train shape:", labels_train.shape)

train_features shape: (5882, 518)
labels_train shape: (5882,)


In [27]:
# Initialize an empty dictionary to store results
results = {}

for i, (estimator_name, estimator, param_grid) in enumerate(zip(log_list, estimator_list, param_list)):
    print(f'Step {i+1}/{len(estimator_list)}: {estimator_name}...')

    # Use GridSearchCV only if 'predefined' is selected
    if model_selection == 'predefined':
        split_indices = np.concatenate([
            np.full(embeddings.train.shape[0], -1),
            np.zeros(embeddings.validation.shape[0])
        ])
        split = PredefinedSplit(split_indices)
        clf = GridSearchCV(
            estimator=estimator,
            param_grid=param_grid,
            cv=split,
            n_jobs=-1,
            refit=True,
            verbose=0,
            error_score='raise'  # Ensures all errors are raised for easier debugging
        )
    else:
        clf = estimator  # Use the estimator directly without parameter tuning

    # Fit the model and evaluate accuracy and recall
    try:
        clf.fit(train_features, labels_train)
        test_acc = clf.score(test_features, all_data.test.labels)
        test_uar = recall_score(all_data.test.labels, clf.predict(test_features), average='macro')

        # Store results in dictionary format
        results[estimator_name] = {
            'test_acc': test_acc,
            'test_uar': test_uar
        }
        print('Done')

    except ValueError as e:
        print(f"Error with {estimator_name}: {e}")
        continue

# Create a DataFrame from the results, rounding to 1 decimal place
results_df = pd.DataFrame(results).applymap(lambda x: round(x * 100, 1))

# Select the model with the highest test accuracy
best_estimator = results_df.loc['test_acc'].idxmax()
results_df = results_df[[best_estimator]].rename(columns={best_estimator: dataset_name})

# Display the final results DataFrame
print(results_df)


Step 1/5: LDA...
Done
Step 2/5: LR...
Error with LR: The 'max_iter' parameter of LogisticRegression must be an int in the range [0, inf). Got 10000.0 instead.
Step 3/5: QDA...
Done
Step 4/5: RF...
Done
Step 5/5: SVC...
Error with SVC: The 'max_iter' parameter of SVC must be an int in the range [-1, inf). Got 10000.0 instead.
          crema_d
test_acc     70.2
test_uar     70.4


<ipython-input-27-8778d0e43743>:44: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  results_df = pd.DataFrame(results).applymap(lambda x: round(x * 100, 1))


In [28]:
# Create a DataFrame from the results, rounding to 1 decimal place
results_df = pd.DataFrame(results).applymap(lambda x: round(x * 100, 1))

# Select the model with the highest test accuracy
best_estimator = results_df.loc['test_acc'].idxmax()
results_df = results_df[[best_estimator]].rename(columns={best_estimator: dataset_name})

# Display the final results DataFrame
print(results_df)

          crema_d
test_acc     70.2
test_uar     70.4


<ipython-input-28-3efe8dfb0e29>:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  results_df = pd.DataFrame(results).applymap(lambda x: round(x * 100, 1))


In [29]:
results_df

,crema_d
test_acc,70.2
test_uar,70.4


In [30]:
filename = os.path.splitext(os.path.basename(weight_file))[0] if weight_file else model_name
results_folder = f'{SERAB_PATH}/clf_results/'
save_results(filename + '.csv', results_df, results_folder)

In [31]:
pd.read_csv(f'{SERAB_PATH}/clf_results/default1024_BYOLAs64x96-2107292000-e100-bs256-lr0003-rs42.csv', index_col=0)

,crema_d,crema_d.1
test_acc,70.2,70.2
test_uar,70.4,70.4
